In [5]:
import numpy as np
import cv2
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D
from keras.optimizers import Adam
from keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
train_dir = 'data/train'
val_dir = 'data/test'
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

In [9]:
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.


In [10]:
validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 7178 images belonging to 7 classes.


In [11]:
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))
# emotion_model.load_weights('emotion_model.h5')

C:\Users\Mansoor\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
cv2.ocl.setUseOpenCL(False)

In [13]:
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

In [ ]:
emotion_model.compile(loss='categorical_crossentropy',
                      optimizer=Adam(learning_rate=0.0001, decay=1e-6),
                      metrics=['accuracy'])

emotion_model_info = emotion_model.fit(
    train_generator,
    steps_per_epoch=28709 // 64,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=7178 // 64
)

emotion_model.save_weights('emotion_model.h5')

Epoch 1/50


C:\Users\Mansoor\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


448/448 ━━━━━━━━━━━━━━━━━━━━ 855s 2s/step - accuracy: 0.2461 - loss: 1.8258 - val_accuracy: 0.3668 - val_loss: 1.6577
Epoch 2/50
  1/448 ━━━━━━━━━━━━━━━━━━━━ 3:33 479ms/step - accuracy: 0.3594 - loss: 1.6367

D:\anaconda\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


448/448 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3594 - loss: 1.6367 - val_accuracy: 0.6000 - val_loss: 1.5658
Epoch 3/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 329s 734ms/step - accuracy: 0.3549 - loss: 1.6456 - val_accuracy: 0.4155 - val_loss: 1.5223
Epoch 4/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 1s 197us/step - accuracy: 0.4375 - loss: 1.4717 - val_accuracy: 0.2000 - val_loss: 1.9958
Epoch 5/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 307s 684ms/step - accuracy: 0.4109 - loss: 1.5326 - val_accuracy: 0.4528 - val_loss: 1.4408
Epoch 6/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 1s 149us/step - accuracy: 0.4062 - loss: 1.3893 - val_accuracy: 0.2000 - val_loss: 1.8662
Epoch 7/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 276s 615ms/step - accuracy: 0.4408 - loss: 1.4556 - val_accuracy: 0.4784 - val_loss: 1.3803
Epoch 8/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 1s 150us/step - accuracy: 0.5000 - loss: 1.2801 - val_accuracy: 0.4000 - val_loss: 1.5009
Epoch 9/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 263s 587ms/step - accuracy: 0.4696 - loss: 1.3933 - val_a

In [ ]:
# start the webcam feed
cap = cv2.VideoCapture(0)
while True:
    # Find haar cascade to draw bounding box around face
    ret, frame = cap.read()
    if not ret:
        break
    bounding_box = cv2.CascadeClassifier('/home/shivam/.local/lib/python3.6/site-packages/cv2/data/haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2gray_frame)
    num_faces = bounding_box.detectMultiScale(gray_frame,scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('Video', cv2.resize(frame,(1200,860),interpolation = cv2.INTER_CUBIC))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()